

# Gradient-Based Optimization Algorithms

This project implements and compares several gradient-based optimization algorithms
commonly used in machine learning and deep learning:

* **Gradient Descent (GD)**:
  
  The basic optimization algorithm that updates parameters in the direction of the
  negative gradient of the loss function. Uses the entire dataset for each update.
  
  - Simple and guaranteed to converge for convex functions with appropriate learning rate
  - Computationally expensive for large datasets
  - Can be slow to converge

* **Stochastic Gradient Descent (SGD)**:
  
  Updates parameters using only one randomly selected sample at a time.
  
  - Much faster updates than GD
  - High variance in updates can lead to noisy convergence
  - Can escape local minima due to noise
  - Requires careful learning rate tuning

* **Mini-Batch Gradient Descent (Batch GD)**:
  
  Compromise between GD and SGD - uses small batches of samples for each update.
  
  - Reduces variance compared to SGD
  - More efficient than full GD
  - Can leverage vectorization for speed
  - Most commonly used in practice

* **Adam (Adaptive Moment Estimation)**:
  
  Combines ideas from RMSprop and momentum. Maintains adaptive learning rates
  for each parameter and uses both first and second moments of gradients.
  
  - Adaptive learning rates per parameter
  - Works well with sparse gradients
  - Requires little tuning
  - Most popular optimizer in deep learning

* **Nesterov Accelerated Gradient (NAG)**:
  
  Improvement over momentum-based methods. Looks ahead by calculating gradient
  at the anticipated future position.
  
  - Better convergence properties than standard momentum
  - Particularly effective for convex functions
  - Reduces oscillations

* **Adan (Adaptive Nesterov Momentum)**:
  
  Recent optimizer combining adaptive learning rates with Nesterov momentum.
  
  - Faster convergence than Adam in many cases
  - More stable training
  - Effective for both convex and non-convex optimization


In [3]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


In [ ]:
# Define test functions for optimization
def rosenbrock(x, y):
    #Rosenbrock function: a classic non-convex optimization test function
    return (1 - x)**2 + 100 * (y - x**2)**2

def rosenbrock_grad(x, y):
    #Gradient of the Rosenbrock function
    dx = -2 * (1 - x) - 400 * x * (y - x**2)
    dy = 200 * (y - x**2)
    return np.array([dx, dy])

def beale(x, y):
    #Beale function: another test function with multiple local minima
    term1 = (1.5 - x + x*y)**2
    term2 = (2.25 - x + x*y**2)**2
    term3 = (2.625 - x + x*y**3)**2
    return term1 + term2 + term3

def beale_grad(x, y):
    #Gradient of the Beale function
    dx = (2 * (1.5 - x + x*y) * (y - 1) +
          2 * (2.25 - x + x*y**2) * (y**2 - 1) +
          2 * (2.625 - x + x*y**3) * (y**3 - 1))
    dy = (2 * (1.5 - x + x*y) * x +
          2 * (2.25 - x + x*y**2) * 2*x*y +
          2 * (2.625 - x + x*y**3) * 3*x*y**2)
    return np.array([dx, dy])

def sphere(x, y):
    #Sphere function: simple convex quadratic function
    return x**2 + y**2

def sphere_grad(x, y):
    #Gradient of the Sphere function
    return np.array([2*x, 2*y])

# Numerical gradient computation for custom functions
def numerical_gradient(func, x, y, h=1e-5):
    grad_x = (func(x + h, y) - func(x - h, y)) / (2 * h)
    grad_y = (func(x, y + h) - func(x, y - h)) / (2 * h)
    return np.array([grad_x, grad_y])

# Function to get custom function from user
def get_custom_function():
    print("\nEnter your custom function f(x, y):")
    print("\nUse np notation")
    func_str = input("\nYour function f(x,y) = ")

    def custom_func(x, y):
        try:
            return eval(func_str)
        except Exception as e:
            print(f"Error evaluating function: {e}")
            return 0
    try:
        test_val = custom_func(1.0, 1.0)
        print(f"Test: f(1, 1) = {test_val}")
    except Exception as e:
        print(f"Error testing function: {e}")
        return None, None

    def custom_grad(x, y):
        return numerical_gradient(custom_func, x, y)

    return custom_func, custom_grad

# Get user choice
print("Available test functions:")
print("1. Rosenbrock function (challenging non-convex)")
print("2. Beale function (multiple local minima)")
print("3. Sphere function (simple convex)")
print("4. Custom function (enter your own)")

function_choice = int(input("\nSelect function (1-4): "))

if function_choice == 1:
    objective_func = rosenbrock
    gradient_func = rosenbrock_grad
    x_range = (-2, 2)
    y_range = (-1, 3)
    start_point = np.array([-1.5, 2.5])
    func_name = "Rosenbrock"
elif function_choice == 2:
    objective_func = beale
    gradient_func = beale_grad
    x_range = (-4.5, 4.5)
    y_range = (-4.5, 4.5)
    start_point = np.array([0.5, 0.5])
    func_name = "Beale"
elif function_choice == 3:
    objective_func = sphere
    gradient_func = sphere_grad
    x_range = (-10, 10)
    y_range = (-10, 10)
    start_point = np.array([8.0, 8.0])
    func_name = "Sphere"
else:  # Custom function
    objective_func, gradient_func = get_custom_function()

    if objective_func is None:
        print("Error with custom function. Using Sphere function instead.")
        objective_func = sphere
        gradient_func = sphere_grad
        x_range = (-10, 10)
        y_range = (-10, 10)
        start_point = np.array([8.0, 8.0])
        func_name = "Sphere"
    else:
        print("\nSet the visualization range:")
        x_min = float(input("x min: "))
        x_max = float(input("x max: "))
        y_min = float(input("y min: "))
        y_max = float(input("y max: "))
        x_range = (x_min, x_max)
        y_range = (y_min, y_max)

        print("\nSet the starting point for optimization:")
        start_x = float(input("Starting x: "))
        start_y = float(input("Starting y: "))
        start_point = np.array([start_x, start_y])
        func_name = "Custom"

# Get optimization parameters with function-specific suggestions
if func_name == "Rosenbrock":
    suggested_lr = "0.0001-0.001"
    default_lr = 0.001
elif func_name == "Beale":
    suggested_lr = "0.001-0.005"
    default_lr = 0.002
elif func_name == "Sphere":
    suggested_lr = "0.01-0.1"
    default_lr = 0.01
else:  # Custom
    suggested_lr = "0.001-0.01"
    default_lr = 0.001

print(f"\nRecommended learning rate for {func_name}: {suggested_lr}")
learning_rate = float(input(f"Enter learning rate (press Enter for {default_lr}): ") or default_lr)
num_iterations = int(input("Enter number of iterations (suggested: 100-1000, press Enter for 500): ") or 500)
batch_size = int(input("Enter batch size for mini-batch GD (suggested: 10-50, press Enter for 32): ") or 32)

In [ ]:
# Objective function plot
plt.figure(figsize=(8, 6))

x = np.linspace(x_range[0], x_range[1], 400)
y = np.linspace(y_range[0], y_range[1], 400)
X, Y = np.meshgrid(x, y)
Z = objective_func(X, Y)

contour = plt.contour(X, Y, Z, levels=40, cmap='viridis')
plt.clabel(contour, inline=True, fontsize=8)

plt.xlabel("x")
plt.ylabel("y")
plt.title(f"Objective Function: {func_name}")
plt.grid(True, alpha=0.3)

plt.show()

## Mathematical Foundations

### General Optimization Problem

We aim to minimize a function $f(\theta)$ where $\theta$ represents the parameters.

$$
\min_{\theta} f(\theta)
$$

---

### Gradient Descent Update Rule

The basic Gradient Descent update is:

$$
\theta_{t+1} = \theta_t - \eta \, \nabla f(\theta_t)
$$

where:

- $\theta_t$ is the parameter vector at iteration $t$
- $\eta$ is the learning rate (step size)
- $\nabla f(\theta_t)$ is the gradient of $f$ evaluated at $\theta_t$

---

### Stochastic Gradient Descent (SGD)

Instead of computing the gradient over the entire dataset, SGD uses a single sample:

$$
\theta_{t+1} = \theta_t - \eta \, \nabla f_i(\theta_t)
$$

where $f_i$ is the loss for the $i$-th sample.

---

### Mini-Batch Gradient Descent

Uses a subset (mini-batch) $B$ of the data:

$$
\theta_{t+1} = \theta_t - \eta \, \frac{1}{|B|} \sum_{i \in B} \nabla f_i(\theta_t)
$$

where $|B|$ is the size of the mini-batch.

---

### Momentum

Adds a velocity term to accelerate convergence:

$$
v_{t+1} = \beta v_t + \nabla f(\theta_t)
$$

$$
\theta_{t+1} = \theta_t - \eta \, v_{t+1}
$$

where $\beta$ is the momentum coefficient (typically $\beta = 0.9$).

---

### Nesterov Accelerated Gradient

Looks ahead before computing the gradient:

$$
v_{t+1} = \beta v_t + \nabla f(\theta_t - \eta \beta v_t)
$$

$$
\theta_{t+1} = \theta_t - \eta \, v_{t+1}
$$

---

### Adam (Adaptive Moment Estimation)

Maintains running averages of both the gradient and its square:

$$
m_t = \beta_1 m_{t-1} + (1-\beta_1)\nabla f(\theta_t)
$$

$$
v_t = \beta_2 v_{t-1} + (1-\beta_2)\left[\nabla f(\theta_t)\right]^2
$$

Bias correction:

$$
\hat{m}_t = \frac{m_t}{1-\beta_1^t}
$$

$$
\hat{v}_t = \frac{v_t}{1-\beta_2^t}
$$

Update rule:

$$
\theta_{t+1} = \theta_t - \eta \, \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}
$$

Typical values: $\beta_1=0.9$, $\beta_2=0.999$, $\epsilon=10^{-8}$.

---

### Adan

Combines adaptive learning with Nesterov momentum and uses gradient differences:

$$
m_t = \beta_1 m_{t-1} + (1-\beta_1)\nabla f(\theta_t)
$$

$$
v_t = \beta_2 v_{t-1} + (1-\beta_2)\left[\nabla f(\theta_t) - \nabla f(\theta_{t-1})\right]
$$

$$
n_t = \beta_3 n_{t-1} + (1-\beta_3)\left[\nabla f(\theta_t) + \beta_2(\nabla f(\theta_t) - \nabla f(\theta_{t-1}))\right]^2
$$

Update rule:

$$
\theta_{t+1} = \theta_t - \eta \, \frac{m_t + \beta_2 v_t}{\sqrt{n_t} + \epsilon}
$$

In [ ]:
## Optimization Algorithm Implementations
def gradient_descent(grad_func, start, lr, iterations, clip_value=10.0):
    #Standard Gradient Descent. Uses the full gradient at each step
    path = [start.copy()]
    theta = start.copy()

    for i in range(iterations):
        grad = grad_func(theta[0], theta[1])
        grad_norm = np.linalg.norm(grad)
        if grad_norm > clip_value:
            grad = grad * (clip_value / grad_norm)

        theta = theta - lr * grad

        if np.any(np.isnan(theta)) or np.any(np.isinf(theta)):
            print(f"Warning: GD diverged at iteration {i}. Stopping early.")
            break

        path.append(theta.copy())

    return np.array(path)

def stochastic_gradient_descent(grad_func, start, lr, iterations, clip_value=10.0):
    # Stochastic Gradient Descent. Adds noise to simulate using single samples
    path = [start.copy()]
    theta = start.copy()

    for i in range(iterations):
        grad = grad_func(theta[0], theta[1])
        grad_norm = np.linalg.norm(grad)
        if grad_norm > clip_value:
            grad = grad * (clip_value / grad_norm)

        noise = np.random.randn(2) * 0.1
        noisy_grad = grad + noise
        theta = theta - lr * noisy_grad

        if np.any(np.isnan(theta)) or np.any(np.isinf(theta)):
            print(f"Warning: SGD diverged at iteration {i}. Stopping early.")
            break

        path.append(theta.copy())

    return np.array(path)

def batch_gradient_descent(grad_func, start, lr, iterations, batch_size, clip_value=10.0):
    # Mini-Batch Gradient Descent. Averages gradients over small batches
    path = [start.copy()]
    theta = start.copy()

    for i in range(iterations):
        batch_grad = np.zeros(2)
        for _ in range(batch_size):
            grad = grad_func(theta[0], theta[1])
            grad_norm = np.linalg.norm(grad)
            if grad_norm > clip_value:
                grad = grad * (clip_value / grad_norm)

            noise = np.random.randn(2) * 0.05
            batch_grad += (grad + noise)
        batch_grad /= batch_size

        theta = theta - lr * batch_grad

        if np.any(np.isnan(theta)) or np.any(np.isinf(theta)):
            print(f"Warning: Mini-Batch GD diverged at iteration {i}. Stopping early.")
            break

        path.append(theta.copy())

    return np.array(path)

def nesterov_gradient_descent(grad_func, start, lr, iterations, momentum=0.9, clip_value=10.0):
    # Nesterov Accelerated Gradient. Looks ahead before computing gradient
    path = [start.copy()]
    theta = start.copy()
    velocity = np.zeros(2)

    for i in range(iterations):
        # Look-ahead position
        theta_ahead = theta - momentum * velocity
        grad = grad_func(theta_ahead[0], theta_ahead[1])

        grad_norm = np.linalg.norm(grad)
        if grad_norm > clip_value:
            grad = grad * (clip_value / grad_norm)

        velocity = momentum * velocity + grad
        theta = theta - lr * velocity

        if np.any(np.isnan(theta)) or np.any(np.isinf(theta)):
            print(f"Warning: Nesterov diverged at iteration {i}. Stopping early.")
            break

        path.append(theta.copy())

    return np.array(path)

def adam_optimizer(grad_func, start, lr, iterations, beta1=0.9, beta2=0.999, epsilon=1e-8, clip_value=10.0):
    #Adam Optimizer. Adaptive learning rate with momentum

    path = [start.copy()]
    theta = start.copy()
    m = np.zeros(2)  # First moment
    v = np.zeros(2)  # Second moment

    for t in range(1, iterations + 1):
        grad = grad_func(theta[0], theta[1])

        grad_norm = np.linalg.norm(grad)
        if grad_norm > clip_value:
            grad = grad * (clip_value / grad_norm)

        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * (grad ** 2)

        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)

        theta = theta - lr * m_hat / (np.sqrt(v_hat) + epsilon)

        if np.any(np.isnan(theta)) or np.any(np.isinf(theta)):
            print(f"Warning: Adam diverged at iteration {t}. Stopping early.")
            break

        path.append(theta.copy())

    return np.array(path)

def adan_optimizer(grad_func, start, lr, iterations, beta1=0.98, beta2=0.92, beta3=0.99, epsilon=1e-8, clip_value=10.0):
    # Adan Optimizer. Adaptive Nesterov momentum with gradient difference
    path = [start.copy()]
    theta = start.copy()
    m = np.zeros(2)  # First moment
    v = np.zeros(2)  # Gradient difference
    n = np.zeros(2)  # Second moment
    grad_prev = grad_func(start[0], start[1])

    for t in range(1, iterations + 1):
        grad = grad_func(theta[0], theta[1])

        grad_norm = np.linalg.norm(grad)
        if grad_norm > clip_value:
            grad = grad * (clip_value / grad_norm)

        m = beta1 * m + (1 - beta1) * grad

        grad_diff = grad - grad_prev
        v = beta2 * v + (1 - beta2) * grad_diff

        combined_grad = grad + beta2 * grad_diff
        n = beta3 * n + (1 - beta3) * (combined_grad ** 2)

        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        n_hat = n / (1 - beta3 ** t)

        theta = theta - lr * (m_hat + beta2 * v_hat) / (np.sqrt(n_hat) + epsilon)

        if np.any(np.isnan(theta)) or np.any(np.isinf(theta)):
            print(f"Warning: Adan diverged at iteration {t}. Stopping early.")
            break

        path.append(theta.copy())

        grad_prev = grad.copy()

    return np.array(path)

# Run all optimizers

path_gd = gradient_descent(gradient_func, start_point, learning_rate, num_iterations)
path_sgd = stochastic_gradient_descent(gradient_func, start_point, learning_rate, num_iterations)
path_batch = batch_gradient_descent(gradient_func, start_point, learning_rate, num_iterations, batch_size)
path_nesterov = nesterov_gradient_descent(gradient_func, start_point, learning_rate, num_iterations)
path_adam = adam_optimizer(gradient_func, start_point, learning_rate, num_iterations)
path_adan = adan_optimizer(gradient_func, start_point, learning_rate, num_iterations)

# Calculate final values and distances to optimum
optimizers = {
    'Gradient Descent': path_gd,
    'SGD': path_sgd,
    'Mini-Batch GD': path_batch,
    'Nesterov': path_nesterov,
    'Adam': path_adam,
    'Adan': path_adan
}

diverged_optimizers = []

for name, path in optimizers.items():
    if len(path) == 0:
        print(f"\n{name}: Failed to start")
        diverged_optimizers.append(name)
        continue

    final_pos = path[-1]

    # Check if optimization diverged
    if np.any(np.isnan(final_pos)) or np.any(np.isinf(final_pos)):
        print(f"\n{name}:")
        print(f"DIVERGED")
        print(f"Completed iterations: {len(path) - 1}")
        diverged_optimizers.append(name)
    else:
        final_value = objective_func(final_pos[0], final_pos[1])
        print(f"\n{name}:")
        print(f"Final position: ({final_pos[0]:.6f}, {final_pos[1]:.6f})")
        print(f"Final function value: {final_value:.6f}")
        print(f"Total iterations: {len(path) - 1}")


In [ ]:
# Visualization
fig = plt.figure(figsize=(20, 12))

# Create grid for contour plot
x = np.linspace(x_range[0], x_range[1], 400)
y = np.linspace(y_range[0], y_range[1], 400)
X, Y = np.meshgrid(x, y)
Z = objective_func(X, Y)

# Plot 1: All paths together
ax1 = plt.subplot(2, 3, 1)
contour = ax1.contour(X, Y, Z, levels=30, alpha=0.6, cmap='viridis')
ax1.clabel(contour, inline=True, fontsize=8)

colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
for (name, path), color in zip(optimizers.items(), colors):
    ax1.plot(path[:, 0], path[:, 1], 'o-', label=name, color=color,
             markersize=3, linewidth=1.5, alpha=0.7)
    ax1.plot(path[0, 0], path[0, 1], 'k*', markersize=15)  # Start point
    ax1.plot(path[-1, 0], path[-1, 1], 'o', color=color, markersize=10)  # End point

ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title(f'All Optimizers on {func_name} Function')
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3)

# Individual plots for each optimizer
plot_configs = [
    (2, 'Gradient Descent', path_gd, 'red'),
    (3, 'Stochastic GD', path_sgd, 'blue'),
    (4, 'Mini-Batch GD', path_batch, 'green'),
    (5, 'Nesterov', path_nesterov, 'orange'),
    (6, 'Adam', path_adam, 'purple')
]

for idx, name, path, color in plot_configs:
    ax = plt.subplot(2, 3, idx)
    contour = ax.contour(X, Y, Z, levels=30, alpha=0.4, cmap='viridis')
    ax.plot(path[:, 0], path[:, 1], 'o-', color=color, markersize=4, linewidth=2)
    ax.plot(path[0, 0], path[0, 1], 'k*', markersize=15, label='Start')
    ax.plot(path[-1, 0], path[-1, 1], 'ro', markersize=10, label='End')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(name)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot 2: Convergence curves
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, (name, path) in enumerate(optimizers.items()):
    ax = axes[idx]

    # Calculate function values along the path
    values = [objective_func(p[0], p[1]) for p in path]

    ax.plot(values, linewidth=2)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Function Value')
    ax.set_title(f'{name} - Convergence')
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')

plt.tight_layout()
plt.show()

# Plot 3: Comparison of convergence speeds
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
for (name, path), color in zip(optimizers.items(), colors):
    values = [objective_func(p[0], p[1]) for p in path]
    plt.plot(values, label=name, color=color, linewidth=2)

plt.xlabel('Iteration')
plt.ylabel('Function Value (log scale)')
plt.title('Convergence Comparison - All Optimizers')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Plot gradient norms
plt.subplot(1, 2, 2)
for (name, path), color in zip(optimizers.items(), colors):
    grad_norms = []
    for p in path:
        grad = gradient_func(p[0], p[1])
        grad_norms.append(np.linalg.norm(grad))
    plt.plot(grad_norms, label=name, color=color, linewidth=2)

plt.xlabel('Iteration')
plt.ylabel('Gradient Norm (log scale)')
plt.title('Gradient Magnitude - All Optimizers')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

plt.tight_layout()
plt.show()


metrics_data = []
for name, path in optimizers.items():
    values = [objective_func(p[0], p[1]) for p in path]
    grad_norms = [np.linalg.norm(gradient_func(p[0], p[1])) for p in path]

    # Find iteration where function value drops below threshold (if applicable)
    threshold = 1.0
    converged_iter = next((i for i, v in enumerate(values) if v < threshold), len(values))

    metrics = {
        'Optimizer': name,
        'Final Value': values[-1],
        'Min Value': min(values),
        'Final Grad Norm': grad_norms[-1],
        'Iterations to threshold': converged_iter
    }
    metrics_data.append(metrics)

    print(f"\n{name}:")
    print(f"Final value: {metrics['Final Value']:.8f}")
    print(f"Minimum value reached: {metrics['Min Value']:.8f}")
    print(f"Final gradient norm: {metrics['Final Grad Norm']:.8f}")
    print(f"Iterations to reach f(x) < {threshold}: {metrics['Iterations to threshold']}")

## Key Observations

Based on the results, we can observe:

1. **Convergence Speed**:
   - Adam and Adan typically converge faster than standard GD
   - Nesterov shows improved convergence over momentum-based methods
   - SGD has noisy but can escape local minima

2. **Stability**:
   - GD and Mini-Batch GD show smooth, stable convergence
   - SGD shows high variance but explores the space well
   - Adam/Adan balance speed and stability

3. **Final Accuracy**:
   - All methods can reach similar final values with proper tuning
   - Adaptive methods (Adam, Adan) require less hyperparameter tuning

4. **Practical Considerations**:
   - Mini-Batch GD is most common in practice (balance of speed and stability)
   - Adam is default choice for many deep learning applications
   - Adan shows promise for faster convergence in recent research
